In [ ]:
# -------------------- 1. Google Drive 마운트 --------------------
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# -------------------- 2. 필요한 라이브러리 설치 --------------------
!pip install ultralytics -q
!pip install deep_sort_realtime -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 87.5 MB/s eta 0:00:00


In [ ]:
# -------------------- 3. 라이브러리 임포트 --------------------
import cv2
import time
import numpy as np
import os
import torch
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# -------------------- 4. 설정 --------------------
# 모델과 영상 경로 (Google Drive 기준)
MODEL_PATH = "/content/drive/MyDrive/YOLO_safety/runs/helmet_detect_exp18/weights/best.pt"
VIDEO_PATH = "/content/drive/MyDrive/YOLO_safety/test_video.mp4"
OUTPUT_VIDEO_PATH = "/content/drive/MyDrive/YOLO_safety/output_test_video.mp4"

# 클래스 정보 (YOLO 학습 시 정의한 순서에 맞게 수정)
CLASS_NAMES = {
    0: "seatbelt_on",
    1: "seatbelt_off",
    2: "lanyard_on",
    3: "lanyard_off",
    4: "helmet_on",
    5: "helmet_off",
}

# 클래스별 색상 정의 (선택)
CLASS_COLORS = {
    0: (0, 255, 0),       # seatbelt_on → 초록
    1: (0, 0, 255),       # seatbelt_off → 빨강
    2: (255, 255, 0),     # lanyard_on → 노랑
    3: (0, 255, 255),     # lanyard_off → 청록
    4: (255, 0, 255),     # helmet_on → 보라
    5: (255, 0, 0),       # helmet_off → 파랑
}

CONF_THRESHOLD = 0.3  # 탐지 신뢰도 기준

# -------------------- 5. 모델 및 비디오 초기화 --------------------
model = YOLO(MODEL_PATH)
cap = cv2.VideoCapture(VIDEO_PATH)

if not cap.isOpened():
    raise IOError("🚨 비디오를 열 수 없습니다. 경로를 확인하세요.")

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

video_writer = cv2.VideoWriter(
    OUTPUT_VIDEO_PATH,
    cv2.VideoWriter_fourcc(*'mp4v'),
    fps,
    (width, height)
)

# -------------------- 6. DeepSORT 추적기 초기화 --------------------
use_gpu = torch.cuda.is_available()
tracker = DeepSort(
    max_age=5,
    embedder='mobilenet',
    half=True,
    bgr=True,
    embedder_gpu=use_gpu
)

# -------------------- 7. 유틸 함수: 박스 그리기 --------------------
def draw_box(frame, bbox, track_id, cls_id):
    label = CLASS_NAMES.get(cls_id, f'class_{cls_id}')
    color = CLASS_COLORS.get(cls_id, (255, 255, 255))  # 기본: 흰색
    x1, y1, x2, y2 = map(int, bbox)
    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
    cv2.putText(frame, f'{label} ID:{track_id}', (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

# -------------------- 8. 메인 처리 루프 --------------------
frame_count = 0
start_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("✅ 영상 종료 또는 프레임 없음")
        break

    # YOLO 탐지
    results = model.predict(frame, conf=CONF_THRESHOLD, verbose=False)[0]

    # 탐지 결과에서 bbox, conf, class 추출
    detections = []
    for box, conf, cls in zip(results.boxes.xyxy, results.boxes.conf, results.boxes.cls):
        x1, y1, x2, y2 = map(int, box)
        detections.append([[x1, y1, x2 - x1, y2 - y1], float(conf), int(cls)])

    # DeepSORT로 추적
    tracks = tracker.update_tracks(detections, frame=frame)

    # 박스 및 추적 ID 표시
    for track in tracks:
        if not track.is_confirmed():
            continue
        tid = track.track_id
        bbox = track.to_ltrb()  # [x1, y1, x2, y2]
        cls_id = int(track.det_class)
        draw_box(frame, bbox, tid, cls_id)

    # FPS 표시
    frame_count += 1
    elapsed = time.time() - start_time
    fps_now = frame_count / elapsed
    cv2.putText(frame, f"FPS: {fps_now:.2f}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

    # 영상 저장
    video_writer.write(frame)

# -------------------- 9. 종료 --------------------
cap.release()
video_writer.release()
cv2.destroyAllWindows()

print(f"🎬 탐지 완료. 결과 저장 위치: {OUTPUT_VIDEO_PATH}")


✅ 영상 종료 또는 프레임 없음
🎬 탐지 완료. 결과 저장 위치: /content/drive/MyDrive/YOLO_safety/output_test_video.mp4


In [ ]:
print(model.names)

{0: 'seatbelt_on', 1: 'seatbelt_off', 2: 'lanyard_on', 3: 'lanyard_off', 4: 'helmet_on', 5: 'helmet_off'}
